<a href="https://colab.research.google.com/github/shahabbadihi/LA/blob/master/TextRank-v3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# TextRank: Bringing Order into Texts
In this part you will implement [TextRank: Bringing Order into Texts](https://web.eecs.umich.edu/~mihalcea/papers/mihalcea.emnlp04.pdf) paper.

[Mihalcea](https://scholar.google.com/citations?user=UetM7FgAAAAJ&hl=en) and [Tarau](https://scholar.google.com/citations?user=JUMRc-oAAAAJ&hl=en) in this paper, introduced TextRank – a **graphbased ranking model for text processing**, and show how it can be successfully used for natural language applications. In particular, they proposed and evaluated two innovative unsupervised approaches for keyword and sentence extraction.

The basic idea implemented by a graph-based ranking model is that of “voting” or “recommendation”.

When one vertex links to another one, it is basically casting a vote for that other vertex. The higher the number of votes that are cast for a vertex, the higher the importance of the vertex. Moreover, the importance of the vertex casting the vote determines how important the vote itself is, and this information is also taken into account by the ranking model.**This paper relies on Google's PageRank**.

## Defenition

Formally, let $G=(V, E)$ be a directed graph with the set of vertices $V$ and set of edges $E$, where $E$ is a subset of $V \times V$. For a given vertex $V_i$ , let $In(V_i)$ be the set of vertices that point to it (predecessors), and let $Out(V_i)$ be the set of vertices that vertex $V_i$ points to (successors). The score of a vertex $V_i$ is defined as follows (Brin and Page, 1998):

$S(V_i) = (1-d) + d* \sum_{j \in In(V_i)} \frac{1}{|Out(V_j)|}S(V_j)$


where d is a damping factor and usually set to 0.85.

## Graph representation

TextRank builds a weighted graph representation of a document using words as nodes and **co-ocurrence** [<sup>1</sup>](#fn1) frequencies between pairs of words as edge weights. It then applies PageRank to this graph, and treats the PageRank score of each word as its significance.

<img src="https://github.com/shahabbadihi/LA/blob/master/textrank.png?raw=1" width="400" align="center">

<span id="fn1"> [1]: In linguistics, co-occurrence or cooccurrence is an above-chance frequency of occurrence of two terms (also known as coincidence or concurrence) from a text corpus alongside each other in a certain order. For example, when the term "strong coffee" appears in a document, the term "espresso bean" probably also tends to occur in that document.</span>


In [ ]:
!wget 'https://github.com/shahabbadihi/LA/raw/master/PageRank.zip'
!7z x '/content/PageRank.zip'

In [2]:
import os
import sys
import copy
import collections

import nltk
import nltk.tokenize

sys.path.append(".")

import pandas
import page_rank
import text_rank

In [3]:
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


True

## Preprocessing
**Tokenization** is a common task in **Natural Language Processing** (NLP). It’s a fundamental step in both traditional NLP methods like Count Vectorizer and Advanced Deep Learning-based architectures like [Transformers](https://www.analyticsvidhya.com/blog/2019/06/understanding-transformers-nlp-state-of-the-art-models/?utm_source=blog&utm_medium=what-is-tokenization-nlp).

This is the process by which a large quantity of text is divided into smaller parts called **tokens**.

Natural Language toolkit has very important module [**NLTK**](https://www.nltk.org/api/nltk.tokenize.html) tokenize sentences which further comprises of sub-modules word tokenize and sentence tokenize.

We use the method [word_tokenize()](https://www.geeksforgeeks.org/python-nltk-nltk-tokenizer-word_tokenize/) to split a sentence into words. Please refer to below word tokenize NLTK example to understand the theory better.
```python
Input: "I love Applied linear algebra! specially the projects."
Output: ['I', 'love', 'Applied', 'linear', 'algebra', '!', 'specially', 'the', 'projects', '.']
```
After tokenizing the document we should filter irrelevant [PoS tags](https://en.wikipedia.org/wiki/Part-of-speech_tagging) and punctuation (e.g, !, ?).

In [29]:
def __preprocess_document(document, relevant_pos_tags):
    '''
    This function accepts a string representation 
    of a document as input, and returns a tokenized
    list of words corresponding to that document.
    '''
    
    # Tokenizing the document
    words = nltk.tokenize.word_tokenize(document)
    # print(document)
    print(words)

    
    # PoS tagging
    # Your code here, use nltk.pos_tag for words and make a list of second pair
    pos_tags = nltk.pos_tag(words)
    pos_tags = [tag for _, tag in pos_tags]
    print('pos_tags: ', pos_tags, end='\n')
    
    # Filter out words with irrelevant POS tags
    filtered_words = []
    for index, word in enumerate(words):
        word = word.lower()

        tag = pos_tags[index]

        # TODO: append `word` to `filtered_words` if the word is not a punctuation and pos is relevant.
        # You can use `__is_punctuation` function and `relevant_pos_tags`
        if (not text_rank.__is_punctuation(word) and tag in relevant_pos_tags):
            filtered_words.append(word)

    return filtered_words


print(__preprocess_document("hello hello", relevant_pos_tags=["NN", "ADJ"]))

['hello', 'hello']
pos_tags:  ['NN', 'NN']
['hello', 'hello']


## Ranking
In this section, first we will implement weighted PageRank and use this function to implement textRank.

### PageRank using Power method

In [72]:
def power_iteration(transition_weights, rsp=0.15, epsilon=0.00001, max_iterations=1000):
    # Clerical work:
    transition_weights = pandas.DataFrame(transition_weights)
    nodes = page_rank.__extract_nodes(transition_weights)
    transition_weights = page_rank.__make_square(transition_weights, nodes, default=0.0)
    transition_weights = page_rank.__ensure_rows_positive(transition_weights)
    # Setup:
    state = page_rank.__start_state(nodes)
    transition_probabilities = page_rank.__normalize_rows(transition_weights)

    # print(type(state))
    
    # Compute transition matrix
    # Your code here
    transition_matrix = transition_probabilities
    cols_indices_all = [col for col in transition_matrix]
    transition_matrix[cols_indices_all] = (transition_matrix[cols_indices_all] * (1 - rsp)) + (rsp / len(nodes))

    
    # Power iteration:
    # TODO: implement power method
    # Use state.copy() for copying to old_state
    for iteration in range(max_iterations):
        new_state = state.dot(transition_matrix)
        l1_norm = new_state.abs().sum()
        state = new_state.copy().divide(l1_norm)
        

    return state

### TextRank algorithm
Authors used a co-occurrence relation (as we discussed), controlled by the distance between word occurrences: **two vertices are connected** if their corresponding lexical units co-occur within a **window of maximum  words**, where  can be set anywhere from 2 to 10 word.


The vertices added to the graph can be restricted with **syntactic filters**, which select only lexical units of a certain part of speech. One can for instance consider only nouns and verbs for addition to the graph, and consequently draw potential edges based only on relations that can be established between nouns and verb. Experiments showed that **best results observed for nouns ("NN") and adjectives ("ADJ") only**.

In [62]:
def textrank(document, window_size=2, rsp=0.15, relevant_pos_tags=["NN", "ADJ"]):
    '''
    Accepts a string representation
    of a document and returns Pandas matrix that maps words to their related TextRank scores.
    Keyword arguments:
    window_size: window of maximum words, can be set between 2 to 10. (default 2)
    rsp:
    relevant_pos_tags: list tags that graph is restricted by (default ["NN", "ADJ"])
    '''
    
    # Tokenize document:
    words = __preprocess_document(document, relevant_pos_tags)
    
    
    # Building the weighted graph:
    # nodes: words
    # edge weights number of times words cooccur within a window of predetermined size
    edge_weights = collections.defaultdict(lambda: collections.Counter())
    for index, word in enumerate(words):
        for other_index in range(index - window_size, index + window_size + 1):
            if other_index >= 0 and other_index < len(words) and other_index != index:
                other_word = words[other_index]
                edge_weights[word][other_word] += 1.0


    # Apply `power_iteration` to `edge_weights` and sort the output
    # Your code here
    word_probabilities = power_iteration(edge_weights)
    word_probabilities.sort_values(ascending=False, inplace=True)

    
    return word_probabilities

## Apply TextRank

In [78]:
def apply_text_rank(file_name, title="a document"):
    print("Reading \"%s\" ..." % title)
    # Opening:
    
    file_path = os.path.join(os.path.abspath(''), file_name)
    document = open(file_path).read()
    document = text_rank.__ascii_only(document)
    
    print("Applying TextRank to \"%s\" ..." % title)
    
    # TODO: get TextRank vector
    keyword_scores = textrank(document)

    print()
    header = "Keyword Significance Scores for \"%s\":" % title
    print(header)
    print("-" * len(header))
    print(keyword_scores)
    print(keyword_scores['wife'])

apply_text_rank("data/Cinderalla.txt", "Cinderalla")

Reading "Cinderalla" ...
Applying TextRank to "Cinderalla" ...
['Once', 'upon', 'a', 'time', ',', 'there', 'lived', 'a', 'gentleman', ',', 'who', 'after', 'his', 'beautiful', 'and', 'kind', 'wife', 'died', ',', 'married', 'the', 'proudest', 'and', 'meanest', 'woman', 'in', 'all', 'the', 'land', '.', 'She', 'had', 'two', 'daughters', 'from', 'a', 'previous', 'marriage', 'who', 'were', 'just', 'as', 'nasty', 'and', 'haughty', 'as', 'their', 'mother', '.', 'The', 'gentleman', 'also', 'had', 'a', 'young', 'daughter', 'by', 'another', 'wife', 'named', 'Cinderella', ',', 'who', 'was', 'filled', 'with', 'goodness', 'and', 'was', 'one', 'of', 'the', 'sweetest', 'girls', 'the', 'kingdom', 'had', 'ever', 'seen', '.', 'Cinderella', "'s", 'stepmother', 'was', 'extremely', 'jealous', 'of', 'her', 'beauty', 'and', 'charm', 'and', 'made', 'her', 'do', 'the', 'hardest', 'and', 'most', 'dreadful', 'work', 'in', 'the', 'house', '.', 'Cinderella', 'did', 'the', 'dishes', ',', 'scrubbed', 'the', 'floor', 

### Cinderalla story

In [ ]:
apply_text_rank("data/Cinderalla.txt", "Cinderalla")

Reading "Cinderalla" ...
Applying TextRank to "Cinderalla" ...

Keyword Significance Scores for "Cinderalla":
---------------------------------------------
time           0.017010
gentleman      0.013592
beautiful      0.008104
kind           0.008308
wife           0.014819
                 ...   
girl           0.006933
forgiveness    0.007110
way            0.007107
heart          0.006917
palace         0.005507
Length: 90, dtype: float64



### Beauty_and_the_Beast story

In [ ]:
apply_text_rank("data/Beauty_and_the_Beast.txt", "Beauty and the Beast")

Reading "Beauty and the Beast" ...
Applying TextRank to "Beauty and the Beast" ...

Keyword Significance Scores for "Beauty and the Beast":
-------------------------------------------------------
time           0.021454
king           0.062943
queen          0.050955
day            0.010592
birth          0.010896
baby           0.037179
girl           0.036224
kingdom        0.042635
celebration    0.010618
fairy          0.032813
well           0.011154
world          0.011714
fairys         0.011781
turn           0.011295
spindle        0.035759
die            0.010347
crying         0.010237
child          0.019615
sleep          0.010400
prince         0.082339
everyone       0.024608
princess       0.074160
kind           0.010918
need           0.011302
everybody      0.011134
lady           0.029866
spinning       0.011324
course         0.011298
moment         0.011194
floor          0.011853
slumber        0.011849
back           0.011170
palace         0.026402
bed         

In [ ]:
# Optional: test textRank on another documents :))